In [24]:
import torch
import torchhd
import torchhd.functional as functional

x = torch.randint(-100, 100, (4, 6))
x

tensor([[-64, -11,   6,  97,  83, -95],
        [-24,  34, -21,  49, -35, -16],
        [-52, -53, -70, -88,  -4,   9],
        [ 50, -35, -46, -83, -45,  20]])

In [25]:
y = torchhd.random(1, 6, "MAP")
y

MAPTensor([[ 1., -1.,  1., -1.,  1., -1.]])

In [26]:
functional.dot_similarity(y, x)

MAPTensor([[  34., -147.,    6.,   57.]])

In [27]:
x_binary = torchhd.normalize(x)
x_binary

MAPTensor([[-1, -1,  1,  1,  1, -1],
           [-1,  1, -1,  1, -1, -1],
           [-1, -1, -1, -1, -1,  1],
           [ 1, -1, -1, -1, -1,  1]])

In [28]:
functional.dot_similarity(y, x_binary)

MAPTensor([[ 2., -4., -2.,  0.]])

# Binary projection Matrix

In [33]:
import torch.nn as nn
import torchhd
from torchhd import embeddings

class Encoder(nn.Module):
    def __init__(self, hd_dim, size):
        super(Encoder, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.projection = embeddings.Projection(size, hd_dim)

    def forward(self, x):
        sample_hv = self.projection(x)
        return torchhd.hard_quantize(sample_hv).to(torch.int32)

In [38]:
encode = Encoder(5000, 7)

In [39]:
x = torch.randn(3, 7)
x

tensor([[-0.3013, -0.6094,  0.1676,  0.9796,  1.7485,  0.7745,  0.3398],
        [-1.8307,  1.8629,  0.8918,  1.6368,  1.0036,  0.1416, -1.1592],
        [-0.6027, -0.0637, -0.3587, -0.5033, -0.4574, -0.0178, -0.2458]])

In [40]:
y = encode(x)
y

/tmp/ipykernel_18853/1322912733.py:13: DeprecationWarning: torchhd.hard_quantize is deprecated, consider using torchhd.normalize instead.
  return torchhd.hard_quantize(sample_hv).to(torch.int32)


MAPTensor([[-1, -1, -1,  ...,  1,  1,  1],
           [ 1, -1, -1,  ..., -1, -1,  1],
           [ 1,  1,  1,  ..., -1, -1,  1]], dtype=torch.int32)

In [44]:
encode.projection.weight = nn.Parameter(torchhd.normalize(encode.projection.weight), requires_grad=False) # Binary

In [46]:
encode.projection.weight.shape

torch.Size([5000, 7])